gensim
http://nbviewer.jupyter.org/gist/boskaiolo/cc3e1341f59bfbd02726
http://blog.csdn.net/u010297828/article/details/50464845



sklearn
http://www.cnblogs.com/pinard/p/6908150.html



In [2]:
from sklearn import datasets
news_dataset = datasets.fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

INFO : Downloading 20news dataset. This may take a few minutes.
INFO : Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


In [3]:
# A list of text document is contained in the data variable
documents = news_dataset.data

print "In the dataset there are", len(documents), "textual documents"
print "And this is the first one:\n", documents[0]

In the dataset there are 18846 textual documents
And this is the first one:


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [8]:
# gensim.parsing.preprocessing.STOPWORDS

In [6]:
def tokenize(text):
    return [token for token in gensim.utils.simple_preprocess(text) if token not in gensim.parsing.preprocessing.STOPWORDS]

print "After the tokenizer, the previous document becomes:\n", tokenize(documents[0])


After the tokenizer, the previous document becomes:
[u'sure', u'bashers', u'pens', u'fans', u'pretty', u'confused', u'lack', u'kind', u'posts', u'recent', u'pens', u'massacre', u'devils', u'actually', u'bit', u'puzzled', u'bit', u'relieved', u'going', u'end', u'non', u'pittsburghers', u'relief', u'bit', u'praise', u'pens', u'man', u'killing', u'devils', u'worse', u'thought', u'jagr', u'showed', u'better', u'regular', u'season', u'stats', u'lot', u'fo', u'fun', u'watch', u'playoffs', u'bowman', u'let', u'jagr', u'lot', u'fun', u'couple', u'games', u'pens', u'going', u'beat', u'pulp', u'jersey', u'disappointed', u'islanders', u'lose', u'final', u'regular', u'season', u'game', u'pens', u'rule']


In [10]:
processed_docs = [tokenize(doc) for doc in documents]

In [19]:
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print "In the corpus there are", len(word_count_dict), "unique tokens"

In the corpus there are 95507 unique tokens


In [20]:
word_count_dict.filter_extremes(no_below=20, no_above=0.1) # word must appear >10 times, and no more than 20% documents

In [21]:
print "After filtering, in the corpus there are only", len(word_count_dict), "unique tokens"

After filtering, in the corpus there are only 8121 unique tokens


In [22]:
bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

In [23]:
bow_doc1 = bag_of_words_corpus[0]

print "Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1
print
for i in range(5):
    print "In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1])
print "..."

Bag of words representation of the first document (tuples are composed by token_id and multiplicity):
[(219, 1), (770, 2), (780, 2), (1354, 1), (1375, 1), (1568, 1), (1723, 2), (2024, 1), (2699, 1), (3194, 1), (3215, 1), (3353, 1), (3467, 1), (3755, 1), (3853, 1), (3880, 1), (3966, 1), (4213, 1), (4304, 2), (4678, 1), (4703, 1), (4840, 1), (4897, 1), (5001, 1), (5243, 5), (5398, 2), (5405, 1), (5454, 2), (5510, 3), (5694, 1), (5877, 1), (5985, 1), (6212, 1), (6273, 1), (6393, 1), (6437, 1), (6677, 1), (6852, 2), (6885, 1), (7031, 1), (7162, 1), (7185, 1), (7370, 1), (7882, 1)]

In the document, topic_id 219 (word "showed") appears 1 time[s]
In the document, topic_id 770 (word "jagr") appears 2 time[s]
In the document, topic_id 780 (word "going") appears 2 time[s]
In the document, topic_id 1354 (word "recent") appears 1 time[s]
In the document, topic_id 1375 (word "couple") appears 1 time[s]
...


In [24]:
# LDA mono-core
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)


In [30]:
lda_model[bag_of_words_corpus[0]]

[(5, 0.023679503768661413),
 (6, 0.41431114779458444),
 (7, 0.017291639785003393),
 (8, 0.51751020260211666),
 (9, 0.018585604973863107)]

In [32]:
sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1])

[(8, 0.51750670596247794),
 (6, 0.41432467490281838),
 (5, 0.023679370917892333),
 (9, 0.01858703155864945),
 (7, 0.017280315593002134)]

In [33]:
 documents[0]

u"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [27]:
for index, score in sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1]):
    print "Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10))

Score: 0.517509557652	 Topic: 0.019*"game" + 0.016*"team" + 0.013*"games" + 0.012*"year" + 0.009*"season" + 0.009*"hockey" + 0.009*"play" + 0.007*"league" + 0.007*"players" + 0.006*"st"
Score: 0.414313056494	 Topic: 0.006*"going" + 0.006*"ll" + 0.006*"car" + 0.006*"got" + 0.006*"said" + 0.004*"little" + 0.004*"thing" + 0.004*"old" + 0.004*"lot" + 0.004*"sure"
Score: 0.0236794807207	 Topic: 0.049*"god" + 0.014*"jesus" + 0.012*"bible" + 0.011*"church" + 0.010*"christ" + 0.010*"christian" + 0.008*"man" + 0.008*"lord" + 0.007*"christians" + 0.006*"sin"
Score: 0.0185857822674	 Topic: 0.014*"db" + 0.010*"st" + 0.010*"mm" + 0.008*"ma" + 0.008*"hz" + 0.007*"mp" + 0.007*"md" + 0.007*"ra" + 0.007*"mv" + 0.007*"lc"
Score: 0.0172902217921	 Topic: 0.012*"government" + 0.009*"key" + 0.007*"law" + 0.006*"public" + 0.005*"president" + 0.005*"mr" + 0.005*"war" + 0.005*"gun" + 0.005*"state" + 0.005*"armenian"


In [34]:
news_dataset.target_names[news_dataset.target[0]]

'rec.sport.hockey'

In [35]:
for index, score in sorted(lda_model[bag_of_words_corpus[1]], key=lambda tup: -1*tup[1]):
    print "Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10))

Score: 0.792241842452	 Topic: 0.011*"windows" + 0.009*"dos" + 0.008*"file" + 0.007*"image" + 0.007*"bit" + 0.006*"drive" + 0.006*"program" + 0.006*"card" + 0.005*"disk" + 0.005*"software"
Score: 0.13329362145	 Topic: 0.026*"edu" + 0.015*"mail" + 0.014*"com" + 0.009*"available" + 0.009*"ftp" + 0.009*"send" + 0.008*"information" + 0.008*"email" + 0.008*"list" + 0.006*"pub"
Score: 0.0532479917408	 Topic: 0.006*"going" + 0.006*"ll" + 0.006*"car" + 0.006*"got" + 0.006*"said" + 0.004*"little" + 0.004*"thing" + 0.004*"old" + 0.004*"lot" + 0.004*"sure"


In [36]:
unseen_document = "In my spare time I either play badmington or drive my car"
print "The unseen document is composed by the following text:", unseen_document
print

bow_vector = word_count_dict.doc2bow(tokenize(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print "Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5))

The unseen document is composed by the following text: In my spare time I either play badmington or drive my car

Score: 0.630699929351	 Topic: 0.006*"going" + 0.006*"ll" + 0.006*"car" + 0.006*"got" + 0.006*"said"
Score: 0.209286907818	 Topic: 0.019*"game" + 0.016*"team" + 0.013*"games" + 0.012*"year" + 0.009*"season"
Score: 0.0200124118895	 Topic: 0.011*"windows" + 0.009*"dos" + 0.008*"file" + 0.007*"image" + 0.007*"bit"
Score: 0.0200003248592	 Topic: 0.006*"believe" + 0.005*"point" + 0.005*"evidence" + 0.005*"question" + 0.004*"israel"
Score: 0.0200001895866	 Topic: 0.011*"space" + 0.005*"program" + 0.005*"high" + 0.005*"research" + 0.004*"earth"
Score: 0.0200001686062	 Topic: 0.012*"government" + 0.009*"key" + 0.007*"law" + 0.006*"public" + 0.005*"president"
Score: 0.0200000678415	 Topic: 0.049*"god" + 0.014*"jesus" + 0.012*"bible" + 0.011*"church" + 0.010*"christ"
Score: 0.0200000000377	 Topic: 0.026*"edu" + 0.015*"mail" + 0.014*"com" + 0.009*"available" + 0.009*"ftp"
Score: 0.0200

In [37]:
print "Log perplexity of the model is", lda_model.log_perplexity(bag_of_words_corpus)

Log perplexity of the model is -7.57361194518


In [41]:
lda_model.components_

AttributeError: 'LdaModel' object has no attribute 'components_'

In [39]:
_

[(0,
  u'0.787*"ax" + 0.059*"max" + 0.009*"pl" + 0.008*"wm" + 0.007*"di" + 0.007*"tm" + 0.006*"ei" + 0.005*"ey" + 0.003*"gk" + 0.003*"um"'),
 (1,
  u'0.026*"edu" + 0.015*"mail" + 0.014*"com" + 0.009*"available" + 0.009*"ftp" + 0.009*"send" + 0.008*"information" + 0.008*"email" + 0.008*"list" + 0.006*"pub"'),
 (2,
  u'0.006*"believe" + 0.005*"point" + 0.005*"evidence" + 0.005*"question" + 0.004*"israel" + 0.004*"religion" + 0.004*"true" + 0.004*"things" + 0.004*"fact" + 0.003*"reason"'),
 (3,
  u'0.011*"windows" + 0.009*"dos" + 0.008*"file" + 0.007*"image" + 0.007*"bit" + 0.006*"drive" + 0.006*"program" + 0.006*"card" + 0.005*"disk" + 0.005*"software"'),
 (4,
  u'0.011*"space" + 0.005*"program" + 0.005*"high" + 0.005*"research" + 0.004*"earth" + 0.004*"health" + 0.004*"data" + 0.004*"center" + 0.004*"year" + 0.004*"nasa"'),
 (5,
  u'0.049*"god" + 0.014*"jesus" + 0.012*"bible" + 0.011*"church" + 0.010*"christ" + 0.010*"christian" + 0.008*"man" + 0.008*"lord" + 0.007*"christians" + 0.006*